In [93]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

In [94]:
# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beema\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\beema\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [95]:
# Load the datasets
datasets = {
    'companies': pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\companies\companies.csv'),
    'company_industries':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\companies\company_industries.csv'),
    'company_specialities':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\companies\company_specialities.csv'),
    'employee_counts':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\companies\employee_counts.csv'),
    'benefits':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\jobs\benefits.csv'),
    'job_industries':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\jobs\job_industries.csv'),
    'job_skills': pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\jobs\job_skills.csv'),
    'salaries':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\jobs\salaries.csv'),
    'industries':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\mappings\industries.csv'),
    'skills':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\mappings\skills.csv'),
    'postings':  pd.read_csv(r'C:\Users\beema\Downloads\Linkedindataset\postings.csv')
}


In [96]:
# Display info and first few rows of all DataFrames
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df.info(), '\n')
    print(df.head(), '\n')
    print('-' * 80)

Dataset: companies
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24473 entries, 0 to 24472
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company_id    24473 non-null  int64  
 1   name          24472 non-null  object 
 2   description   24176 non-null  object 
 3   company_size  21699 non-null  float64
 4   state         24451 non-null  object 
 5   country       24473 non-null  object 
 6   city          24472 non-null  object 
 7   zip_code      24445 non-null  object 
 8   address       24451 non-null  object 
 9   url           24473 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.9+ MB
None 

   company_id                        name  \
0        1009                         IBM   
1        1016               GE HealthCare   
2        1025  Hewlett Packard Enterprise   
3        1028                      Oracle   
4        1033                   Accenture   

                

In [97]:
# Find missing values
def find_missing_values(df):
    return df.isnull().sum()

# Check for missing values in each dataset
for name, df in datasets.items():
    print(f"Missing values in {name}:")
    print(find_missing_values(df))
    print()

Missing values in companies:
company_id         0
name               1
description      297
company_size    2774
state             22
country            0
city               1
zip_code          28
address           22
url                0
dtype: int64

Missing values in company_industries:
company_id    0
industry      0
dtype: int64

Missing values in company_specialities:
company_id    0
speciality    0
dtype: int64

Missing values in employee_counts:
company_id        0
employee_count    0
follower_count    0
time_recorded     0
dtype: int64

Missing values in benefits:
job_id      0
inferred    0
type        0
dtype: int64

Missing values in job_industries:
job_id         0
industry_id    0
dtype: int64

Missing values in job_skills:
job_id       0
skill_abr    0
dtype: int64

Missing values in salaries:
salary_id                0
job_id                   0
max_salary            6838
med_salary           33947
min_salary            6838
pay_period               0
currency          

companies have missing values in 7 columns name, description, company_size, state, city, zip_code, address
salaries have missing values in 3: max_salary, med_salary, min_salary
industries have missing values in industry_name    
postings have missing values in 17 columns                      

In [98]:
# Handle missing values in companies dataset
companies = datasets['companies']
companies['name'].fillna('Unknown', inplace=True)
companies['description'].fillna('', inplace=True)
companies['company_size'].fillna(companies['company_size'].median(), inplace=True)
companies.dropna(subset=['state'], inplace=True)
companies.dropna(subset=['city'], inplace=True)
companies.dropna(subset=['zip_code'], inplace=True)
companies['address'].fillna('Unknown', inplace=True)

# Handle missing values in salaries dataset
salaries = datasets['salaries']
salaries['max_salary'].fillna(salaries['max_salary'].mean(), inplace=True)
salaries['min_salary'].fillna(salaries['min_salary'].mean(), inplace=True)
salaries.drop(columns=['med_salary'], inplace=True)

# Handle missing values in industries dataset
industries = datasets['industries']
industries.dropna(subset=['industry_name'], inplace=True)

# Handle missing values in postings dataset
postings = datasets['postings']
postings['company_name'].fillna('Unknown', inplace=True)
postings['description'].fillna('', inplace=True)
postings['max_salary'].fillna(postings['max_salary'].mean(), inplace=True)
postings['min_salary'].fillna(postings['min_salary'].mean(), inplace=True)
postings['pay_period'].fillna(postings['pay_period'].mode()[0], inplace=True)
postings['currency'].fillna(postings['currency'].mode()[0], inplace=True)
postings['compensation_type'].fillna(postings['compensation_type'].mode()[0], inplace=True)
postings.dropna(subset=['company_id'], inplace=True)
postings['views'].fillna(0, inplace=True)
postings.drop(columns=['med_salary'], inplace=True)
postings['applies'].fillna(0, inplace=True)
postings['remote_allowed'].fillna(0, inplace=True)
postings['application_url'].fillna('Unknown', inplace=True)
postings.drop(columns=['closed_time'], inplace=True)
postings['formatted_experience_level'].fillna(postings['formatted_experience_level'].mode()[0], inplace=True)
postings['skills_desc'].fillna('', inplace=True)
postings['posting_domain'].fillna('Unknown', inplace=True)

# Confirm changes
for name, df in datasets.items():
    print(f"Missing values in {name}:")
    print(df.isnull().sum())
    print()

Missing values in companies:
company_id      0
name            0
description     0
company_size    0
state           0
country         0
city            0
zip_code        0
address         0
url             0
dtype: int64

Missing values in company_industries:
company_id    0
industry      0
dtype: int64

Missing values in company_specialities:
company_id    0
speciality    0
dtype: int64

Missing values in employee_counts:
company_id        0
employee_count    0
follower_count    0
time_recorded     0
dtype: int64

Missing values in benefits:
job_id      0
inferred    0
type        0
dtype: int64

Missing values in job_industries:
job_id         0
industry_id    0
dtype: int64

Missing values in job_skills:
job_id       0
skill_abr    0
dtype: int64

Missing values in salaries:
salary_id            0
job_id               0
max_salary           0
min_salary           0
pay_period           0
currency             0
compensation_type    0
dtype: int64

Missing values in industries:
indus

In [92]:
# Define function for text preprocessing
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and non-alphanumeric tokens
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Apply the preprocessing function to the relevant text columns in datasets
datasets['companies']['description'] = datasets['companies']['description'].apply(preprocess_text)
datasets['postings']['description'] = datasets['postings']['description'].apply(preprocess_text)


In [99]:
# Merge company_industries with companies
company_industries = datasets['company_industries']
companies = datasets['companies'].merge(company_industries, on='company_id', how='left')

# Merge company_specialities with companies
company_specialities = datasets['company_specialities']
companies = companies.merge(company_specialities, on='company_id', how='left')

# Merge employee_counts with companies
employee_counts = datasets['employee_counts']
companies = companies.merge(employee_counts, on='company_id', how='left')

# Update the datasets dictionary
datasets['companies'] = companies


In [100]:
# Merge job_industries with postings
job_industries = datasets['job_industries']
postings = datasets['postings'].merge(job_industries, on='job_id', how='left')

# Merge job_skills with postings
job_skills = datasets['job_skills']
postings = postings.merge(job_skills, on='job_id', how='left')

# Merge salaries with postings
salaries = datasets['salaries']
postings = postings.merge(salaries, on='job_id', how='left')

# Merge benefits with postings
benefits = datasets['benefits']
postings = postings.merge(benefits, on='job_id', how='left')

# Update the datasets dictionary
datasets['postings'] = postings


In [102]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode Categorical Variables
label_encoders = {}

# Encode categorical columns in companies dataset
for column in ['name', 'description', 'state', 'country', 'city', 'address']:
    if column in companies.columns:
        le = LabelEncoder()
        companies[column] = le.fit_transform(companies[column])
        label_encoders[column] = le

# Encode categorical columns in company_industries dataset
for column in ['industry']:
    if column in company_industries.columns:
        le = LabelEncoder()
        company_industries[column] = le.fit_transform(company_industries[column])
        label_encoders[column] = le

# Encode categorical columns in skills dataset
for column in ['skill_name']:
    if column in skills.columns:
        le = LabelEncoder()
        skills[column] = le.fit_transform(skills[column])
        label_encoders[column] = le

# Encode categorical columns in industries dataset
for column in ['industry_name']:
    if column in industries.columns:
        le = LabelEncoder()
        industries[column] = le.fit_transform(industries[column])
        label_encoders[column] = le

# Encode categorical columns in postings dataset
for column in ['company_name', 'title', 'description', 'pay_period', 'location', 'formatted_experience_level', 'skills_desc', 'posting_domain', 'compensation_type']:
    if column in postings.columns:
        le = LabelEncoder()
        postings[column] = le.fit_transform(postings[column])
        label_encoders[column] = le

In [103]:
# Feature Engineering: Extract job location features from address
if 'state' in companies.columns and 'city' in companies.columns:
    companies['location'] = companies['state'].astype(str) + '_' + companies['city'].astype(str)
    le = LabelEncoder()
    companies['location'] = le.fit_transform(companies['location'])
    label_encoders['location'] = le

In [104]:
# Scale Numerical Features
scaler = StandardScaler()

# Scale numerical columns in companies dataset
if 'company_size' in companies.columns:
    companies[['company_size']] = scaler.fit_transform(companies[['company_size']])

# Scale numerical columns in postings dataset
for column in ['max_salary', 'min_salary', 'views', 'applies']:
    if column in postings.columns:
        postings[[column]] = scaler.fit_transform(postings[[column]])